In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from colorama import Fore, Back, Style
from tqdm import tqdm_notebook as tqdm
from ipywidgets import Button, Layout
from IPython.display import HTML
import base64
output = widgets.Output()
newseq = []
newnewseq = []
pd.set_option('display.max_colwidth', 0)

In [ ]:
%%html
<h1><u><span style="color:purple">  THE PROTEIN TRUNCATOR TOOL </span></h1> </u>

In [13]:
%%html
<ul>
<li><b> <span style="color:purple"> PLEASE INPUT ONE SEQUENCE e.g GLNDIFEAQKIEWHE</b> </span> </li>
<li><b> <span style="color:purple"> THE TOOL IS NOT SENSITIVE TO CAPITALIZATION. YOU CAN INPUT AHGKL or ahgkl. TOOL WILL UNDERSTAND.   </b> </span> </li>
<li><b> <span style="color:purple"> THE TOOL ONLY UNDERSTANDS ONE LETTER AMINO ACID CODE </b> </span> </li>
<li><b> <span style="color:purple"> ALL SPACES BETWEEN THE AMINO ACIDS WILL BE REMOVED </b> </span> </li>
<li><b> <span style="color:purple"> THE TOOL WILL TELL YOU IF IT DETECTS ANY NON-AMINO ACID SYMBOL. </b> </span> </li>
<li><b> <span style="color:purple"> PLEASE INPUT MASS IN DALTONS e.g ENTER 236000 if MASS of YOUR PEAK is 236 kDA </b> </span> </li>
<li><b> <span style="color:purple"> ALL NON-AMINO ACID SYMBOLS ARE AUTOMATICALLY REMOVED. e.g HGJK becomes HGK </b> </span> </li>
</ul>

In [14]:
%%html
<span style="color:purple"><b> WHAT WILL BE THE OUTPUT? </b> </span>
<ul>
<li><span style="color:purple"><b> The table of the first 50 fragments that have the mass closest to the peak mass </b> </span></li>
<li><span style="color:purple"><b> If there are less than 50 fragments in total all fragments will be shown</b> </span></li>
<li><span style="color:purple"><b> The link to download all fragments within the range that you specified </b> </span></li>

In [ ]:
molecular_weight = {
    'A': 71.0788,
    'E': 129.1155,
    'R': 156.1875,
    'N': 114.1038,
    'D': 115.0886,
    'C': 103.1388, 
    'Q': 128.1307, 
    'G': 57.0519,
    'H': 137.1411,
    'I': 113.1594,
    'L': 113.1594,
    'M': 131.1926,
    'F': 147.1766,
    'P': 97.1167,
    'S': 87.0782,
    'T': 101.1051,
    'W': 186.2132,
    'Y': 163.1760,
    'V': 99.1326,
    'K': 128.1741, 
}
allowed_aa_list = ["A", "R", "N", "D", "C", "Q", "E", "G","H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]

In [ ]:
@output.capture(clear_output=True,wait=True) # based on https://github.com/jupyter-widgets/ipywidgets/issues/1846 and https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
def savesequence(b):
    p=0
    weight = 18.01524
    filename = 'results.xlsx'
    # weight = 18.01524
    title = "Click here to download"
    name = str(nam.value)
    sequences = str(sequen.value)
    Peak = float(your_peak.value)
    #prop = int(y.value)
    prop = y.value
    cutofv = int(Peak/186.2132)
    maxim = int(Peak/57.0519)
    if Peak < 57.0519:
        print("✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵ WARNING ✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵")
        print("The mass of your peak is less than that of single Glycine (G) aminoacid")
        print("Minimum mass of the peak that you can enter is 57.0519 Da")
        print("✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵✵")
    sequences = sequences.upper()
    sequences = list(sequences.split(";"))
    for sequence in sequences:
        x = sequence.replace(' ', '')
        newseq.append(x)

    for sequence in newseq:
        if sequence == '':
            newseq.remove('')
    for sequence in newseq:
        p+=1
        for aa in sequence:
            if aa not in allowed_aa_list:
                print(f"in the sequence {p} we found non-amino acid symbol: {aa}")
                print("we removed it")
                sequence = sequence.replace(aa, "")
        newnewseq.append(sequence)
    #print(newnewseq)
    results = [['positions','MW_Da','Target name', 'Result sequence']]

# So that we can have more than one sequence

    for sequence in newnewseq:
        for aa_position in tqdm(range(0, len(sequence)+1)):
            for i in range(aa_position, len(sequence)+1):
                #positions = (aa_position+1, i)
                position1 = aa_position+1
                position2 = i
                fragment = sequence[aa_position:i]
                
                if len(fragment) >= cutofv and len(fragment) <= maxim:
                    weight = 18.01524
                    fragment_start = fragment[0]
                    fragment_end = fragment[-1]
                    pos_sum = f"{fragment_start}{position1} - {fragment_end}{position2}"
                    for aa in fragment:
                        weight += molecular_weight[aa]
                        
                #print(fragment, weight)
                
                # Saving the results
                    results.append([pos_sum, weight,name, fragment])
    final = pd.DataFrame(results).T.set_index(0).T
    final.insert(2, "Difference from peak", Peak-final['MW_Da'])
    final.insert(3,"Difference from peak module", abs(Peak-final['MW_Da']))
    final_sorted = final.sort_values('Difference from peak module', ascending = True)
    very_final_sorted = final_sorted[final_sorted['Difference from peak module'] <= prop]
    very_final_sorted1 = very_final_sorted.reset_index(drop=True)
    # putting the results into the dataframe
    very_final_sorted1.to_excel('results.xlsx')
    print("FIRST 50 '(OR LESS)' FRAGMENTS")
    display(very_final_sorted1.head(50))
    
   # def create_download_link(filename, title = "Click here to download: "):  
    data = open(filename, "rb").read()
    b64 = base64.b64encode(data)
    payload = b64.decode()
    
    html = '<a download="results.xlsx" href="data:text/xlsx;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title+f' {filename}',filename=filename)
    display(HTML(html))
print("Input your sequence in the window below")
sequen = widgets.Text(placeholder='Input your sequence please')
display(sequen)
print("Input mass of your peak")
your_peak = widgets.Text(placeholder='Input the mass of your observed fragment')
display(your_peak)
print("Input the name of your target")
nam = widgets.Text(placeholder = 'Input the name of the target')
display(nam)
print("Set your maximum difference from peak in daltons (Da)... Default is 10 Da")
print("Maximum is 1000 Da (1kDa). Type or use slider")
dif = widgets.FloatText()
y = widgets.FloatSlider(value = 10, min=0.5, max=1000, step=0.5,description='Difference', disabled=False, continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)
display(dif, y)
mylink = widgets.jslink((y, 'value'), (dif, 'value'))
run_button = widgets.Button(
      description = 'Submit your sequence',layout=Layout(width='18%', height='45px', border='5px solid purple')
)

run_button.on_click(savesequence)
display(run_button)
output

In [7]:
%%html 
<span style ="color:purple"> To start from scratch refresh the window or use the <b> RESET BUTTON  ➔  </b></span>
<button onClick="window.location.reload();">Reset</button>